In [1]:
import nltk; nltk.download('stopwords')
!python3 -m spacy download en
!pip install pyLDAvis
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.0 MB 5.3 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.7 MB 5.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=0b3d89cd37700555dae3a2cc74563e7a20

True

In [2]:
!wget http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
!unzip -q mallet-2.0.8.zip
!mallet-2.0.8/bin/mallet

--2022-06-10 23:43:32--  http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Resolving mallet.cs.umass.edu (mallet.cs.umass.edu)... 128.119.246.70
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip [following]
--2022-06-10 23:43:32--  https://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
Connecting to mallet.cs.umass.edu (mallet.cs.umass.edu)|128.119.246.70|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16184794 (15M) [application/zip]
Saving to: ‘mallet-2.0.8.zip’

mallet-2.0.8.zip    100%[===================>]  15.43M  44.7MB/s    in 0.3s    

2022-06-10 23:43:32 (44.7 MB/s) - ‘mallet-2.0.8.zip’ saved [16184794/16184794]

Unrecognized command: 
Mallet 2.0 commands: 

  import-dir         load the contents of a directory into mallet instances (one per file)
  import-file        load a single file into mallet instances (

In [3]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy
nlp = spacy.load("en_core_web_sm")

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [4]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])
#You should update this path as per the path of Mallet directory on your system.
mallet_path = 'mallet-2.0.8/bin/mallet'

In [5]:
df = pd.read_csv('/content/drive/MyDrive/result.csv')
#df.drop('Unnamed:0')
df

,Unnamed: 0,id,speech,display_as,party,constituency,mnis_id,date,time,colnum,...,RE,Verb,RE2,Pres,topic,politics_phase,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords
0,0,uk.org.publicwhip/debate/1979-11-27a.1096.7,Is the Minister aware that in the last two mon...,Robert Maclennan,Labour,Caithness and Sutherland,578.0,1979-11-27,NaN,1096.0,...,"[('shells', 'vessels', 'Is'), ('they', 'last y...","[been, have, alarmed, landed, entered, appear,...","[('shells', 'vessels', 'Is'), ('they', 'last y...","['in the last two months', 'from my constituen...",NaN,"[[], [], []]",0,8.0,0.1397,"industry, company, make, market, british, busi..."
1,1,uk.org.publicwhip/debate/1979-11-27a.1136.2,Is not the right hon. Gentleman contradicting ...,Roger Moate,Conservative,Faversham,922.0,1979-11-27,16:40:00,1136.0,...,"[('', '', 'Is'), ('Gentleman', 'himself', 'con...","[contradicting, envisages, described, told, co...","[('', '', 'Is'), ('Gentleman', 'himself', 'con...",['on what he has described as the good routes'...,NaN,"[[], [], [], [], [], []]",1,0.0,0.1437,"local, service, area, authority, constituency,..."
2,2,uk.org.publicwhip/debate/1979-11-29a.1477.7,Does the right hon. Gentleman recall that he s...,James Lamond,Labour,Oldham East,894.0,1979-11-29,NaN,1477.0,...,"[('right hon', '', 'Does'), ('colour', 'Britis...","[Does, said, fishing, regarded, reconcile, pre...","[('right hon', '', 'Does'), ('colour', 'Britis...","['for immigrant votes', 'as equal', 'of colour...",NaN,"[[], [], [], []]",2,3.0,0.1612,"country, people, government, political, agree,..."
3,3,uk.org.publicwhip/debate/1979-11-12a.1080.3,"Question put, That the amendment be made:",Unknown,NaN,NaN,NaN,1979-11-12,00:30:00,1080.0,...,"[('', 'amendment', 'put')]","[put, made]","[('', 'amendment', 'put')]",[],NaN,[[]],3,5.0,0.1214,"bill, make, order, amendment, clause, power, l..."
4,4,uk.org.publicwhip/debate/1979-11-29a.1470.2,At a most recent census in August 1979 the Uni...,Alick Buchanan-Smith,Conservative,North Angus and Mearns,727.0,1979-11-29,NaN,1470.0,...,"[('breeding herd', 'pig', 'was')]",[],"[('breeding herd', 'pig', 'was')]","['At a most recent census in August 1979', 'in...",NaN,[[]],4,4.0,0.1172,"case, public, law, police, issue, individual, ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,uk.org.publicwhip/debate/2021-02-24a.1024.2,"Motion made, and Question put forthwith (Stand...",Unknown,NaN,NaN,NaN,2021-02-24,18:56:00,1024.0,...,"[('Question', '83A(7', '\n '), ('following pro...","[made, put, Standing, following, apply, supple...","[('Question', '83A(7', '\n '), ('following pro...",['to the Telecommunications Infrastructure ( L...,1.0,"[[], ['Programme ', 'Programme ']]",9995,5.0,0.2410,"bill, make, order, amendment, clause, power, l..."
9996,9996,uk.org.publicwhip/debate/2021-04-28c.361.4,I have regular discussions with ministerial co...,David Duguid,Conservative,Banff and Buchan,4606.0,2021-04-28,NaN,361.0,...,"[('COP26', 'Scotland', 'have'), ('that', 'Unit...","[have, offers, devolved, brings, devolved, sup...","[('COP26', 'Scotland', 'have'), ('that', 'Unit...","['with ministerial colleagues', 'as well as', ...",NaN,"[[], []]",9996,3.0,0.1930,"country, people, government, political, agree,..."
9997,9997,uk.org.publicwhip/debate/2021-02-22b.655.7,I am really grateful to my hon. Friend for his...,Boris Johnson,Conservative,Uxbridge and South Ruislip,1423.0,2021-02-22,15:51:00,655.0,...,"[('I', 'hon', 'am'), ('', 'question', 'Friend'...","[support, suffered, distributed, put, focus, f...","[('I', 'hon', 'am'), ('', 'question', 'Friend'...","['to my hon', 'for his question', 'to their le...",NaN,"[[], [], [], [], [], []]",9997,9.0,0.1381,"work, important, friend, support, ensure, make..."
9998,9998,uk.org.publicwhip/debate/2021-02-04b.1186.0,I welcome this debate and thank my hon. Friend...,Robert Halfon,Conservative,Harlow,3985.0,2021-02-04,15:45:00,1186.0,...,"[('I', 'hon', 'welcome'), ('', 'town f

In [6]:
data = df.speech.values.tolist()

def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations
#content 变成了列表，然后对列表进行一个处理
data_words = list(sent_to_words(data))

print(data_words[:1])

[['is', 'the', 'minister', 'aware', 'that', 'in', 'the', 'last', 'two', 'months', 'there', 'have', 'been', 'episodes', 'which', 'have', 'alarmed', 'fishermen', 'from', 'my', 'constituency', 'because', 'shells', 'have', 'landed', 'within', 'yards', 'of', 'their', 'vessels', 'is', 'he', 'aware', 'that', 'arrangements', 'entered', 'into', 'last', 'year', 'appear', 'not', 'to', 'be', 'working', 'as', 'they', 'might', 'will', 'he', 'examine', 'this', 'urgently']]


In [7]:
#创建bigram 和trigram 模型，这个可以说明哪些词经常在一起出现使用
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
  
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

#按顺序调用函数
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN'])

print(data_lemmatized[:1])

[['month', 'episode', 'fisherman', 'constituency', 'shell', 'yard', 'vessel', 'arrangement', 'year']]


In [9]:
#创建LDA model
#LDA主题模型的两个主要输入是字典（id2word）和语料库
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]]


In [10]:
def trainmodel(topic):
  ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=topic, id2word=id2word)
  return ldamallet

In [11]:
from gensim.models.wrappers import ldamallet
# Show Topics ---some error with this 
ldamallet= trainmodel(9)
pprint(ldamallet.show_topics(num_words=10))
#output files
ldamallet.save("/content/drive/MyDrive/topic9/mallet_model.model")
new_model = gensim.models.ldamodel.LdaModel.load("/content/drive/MyDrive/topic9/mallet_model.model")

# # Compute Coherence Score
# coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
# coherence_ldamallet = coherence_model_ldamallet.get_coherence()
# print('\nCoherence Score: ', coherence_ldamallet)

[(0,
  '0.078*"friend" + 0.029*"point" + 0.025*"report" + 0.023*"interest" + '
  '0.021*"work" + 0.017*"regulation" + 0.017*"change" + 0.017*"number" + '
  '0.016*"part" + 0.015*"state"'),
 (1,
  '0.086*"people" + 0.039*"child" + 0.030*"work" + 0.030*"school" + '
  '0.024*"year" + 0.023*"family" + 0.019*"support" + 0.018*"education" + '
  '0.015*"time" + 0.014*"number"'),
 (2,
  '0.039*"industry" + 0.036*"government" + 0.031*"business" + 0.028*"policy" + '
  '0.028*"company" + 0.024*"job" + 0.019*"investment" + 0.019*"year" + '
  '0.018*"economy" + 0.017*"market"'),
 (3,
  '0.056*"country" + 0.032*"community" + 0.030*"people" + 0.023*"area" + '
  '0.018*"part" + 0.017*"development" + 0.016*"world" + 0.013*"problem" + '
  '0.012*"agreement" + 0.011*"organisation"'),
 (4,
  '0.178*"member" + 0.053*"friend" + 0.047*"debate" + 0.045*"house" + '
  '0.027*"party" + 0.026*"opposition" + 0.024*"time" + 0.023*"parliament" + '
  '0.016*"today" + 0.015*"motion"'),
 (5,
  '0.060*"service" + 0.022*

In [12]:
from gensim.models.ldamodel import LdaModel
def convertldaMalletToldaGen(mallet_model):
    model_gensim = LdaModel(
        id2word=mallet_model.id2word, num_topics=mallet_model.num_topics,
        alpha=mallet_model.alpha) # original function has 'eta=0' argument
    model_gensim.state.sstats[...] = mallet_model.wordtopics
    model_gensim.sync_state()
    return model_gensim

ldagensim = convertldaMalletToldaGen(ldamallet)

In [13]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(ldagensim, corpus, id2word)
vis

/usr/local/lib/python3.7/dist-packages/pyLDAvis/_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  by='saliency', ascending=False).head(R).drop('saliency', 1)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
6     -0.199240  0.109011       1        1  11.745750
2     -0.122816  0.128739       2        1  11.553069
1     -0.162918  0.020779       3        1  11.360350
5     -0.128143 -0.132907       4        1  11.290889
3     -0.080929 -0.124485       5        1  11.228743
7      0.119871 -0.318418       6        1  11.138923
0      0.187172  0.059130       7        1  10.767715
4      0.270825  0.070615       8        1  10.524242
8      0.116178  0.187535       9        1  10.390318, topic_info=           Term         Freq        Total Category  logprob  loglift
123      member  5738.000000  5738.000000  Default  30.0000  30.0000
44   government  6386.000000  6386.000000  Default  29.0000  29.0000
25       friend  4024.000000  4024.000000  Default  28.0000  28.0000
82       people  4603.000000  4603.000000  Default  27.0000  27.0000
134     service  1923.000000  1923.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
88    situation   216.407851   489.095924   Topic9  -4.9444   1.4489
151      change   274.152856   966.875641   Topic9  -4.7078   1.0039
440        fact   255.556994   931.478295   Topic9  -4.7781   0.9710
191        time   242.833514  2959.465800   Topic9  -4.8291  -0.2361
170        hope   208.578009   609.348496   Topic9  -4.9812   1.1922

[508 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1981      3  0.080428     access
1981      4  0.375330     access
1981      5  0.542143     access
463       4  0.990300   accident
623       1  0.221841    account
...     ...       ...        ...
8         1  0.472824       year
8         2  0.167072       year
8         3  0.211843       year
8         4  0.148235       year
371       9  0.997348  yesterday

[730 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 3, 2, 6, 4, 8, 1, 5, 9])

In [14]:
pyLDAvis.save_html(vis, '/content/drive/MyDrive/topic9/ldamallet.html')

In [19]:
y = ldamallet.print_topics(num_words=50)
modified_format = []
for i in y:
  tmp = []
  tmp.append(i[0])
  tmp.append(i[1].split('+'))
  modified_format.append(tmp)
print(modified_format)
topic_keywords = []
for i in modified_format:
  for j in i[1]:
    topic_keywords_weight=[]
    topic_keywords_weight.append(i[0])
    y = j.split('*')
    topic_keywords_weight.append(float(y[0]))
    topic_keywords_weight.append(y[1].replace('"', ''))
    topic_keywords.append(topic_keywords_weight)

topic_keywords_weight=pd.DataFrame(topic_keywords, columns=['topic', 'fre','keywords'])    
topic_keywords_weight 
data = topic_keywords_weight.infer_objects()
data.to_csv('/content/drive/MyDrive/topic9/keywords_weight_50.csv')

[[0, ['0.078*"friend" ', ' 0.029*"point" ', ' 0.025*"report" ', ' 0.023*"interest" ', ' 0.021*"work" ', ' 0.017*"regulation" ', ' 0.017*"change" ', ' 0.017*"number" ', ' 0.016*"part" ', ' 0.015*"state" ', ' 0.015*"member" ', ' 0.011*"system" ', ' 0.011*"defence" ', ' 0.011*"time" ', ' 0.010*"reason" ', ' 0.010*"account" ', ' 0.009*"matter" ', ' 0.008*"basis" ', ' 0.008*"proposal" ', ' 0.008*"intervention" ', ' 0.008*"position" ', ' 0.007*"control" ', ' 0.007*"contract" ', ' 0.007*"programme" ', ' 0.007*"approach" ', ' 0.007*"deal" ', ' 0.007*"requirement" ', ' 0.007*"advice" ', ' 0.006*"bank" ', ' 0.006*"standard" ', ' 0.006*"option" ', ' 0.005*"moment" ', ' 0.005*"stage" ', ' 0.005*"review" ', ' 0.005*"department" ', ' 0.005*"force" ', ' 0.005*"process" ', ' 0.005*"suggestion" ', ' 0.005*"management" ', ' 0.005*"difficulty" ', ' 0.004*"recommendation" ', ' 0.004*"context" ', ' 0.004*"form" ', ' 0.004*"practice" ', ' 0.004*"doubt" ', ' 0.004*"responsibility" ', ' 0.004*"possibility" ',

In [ ]:
def format_topics_sentences(ldamodel, corpus, texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

df_topic_sents_keywords = format_topics_sentences(ldamodel=ldamallet, corpus=corpus, texts=data_lemmatized)
#df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data_lemmatized)
# Format

df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.to_csv('/content/drive/MyDrive/topic6/df_dominant_topic.csv')

In [20]:
import pickle
topic_distributions = []
length = len(corpus)
for l in range(length):
  tmp = []
  tmp.append(l)
  y = ldamallet[corpus[l]]
  for i in y:
    tmp.append(i[1])
  topic_distributions.append(tmp)
with open('/content/drive/MyDrive/topic9/topic_distribution.pkl', 'wb') as f:
  pickle.dump(topic_distributions, f)

In [1]:
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/result.csv')
df_dominant_topic = pd.read_csv('/content/drive/MyDrive/topic9/df_dominant_topic.csv')
embedding = pd.read_csv('/content/drive/MyDrive/topic9/embedding.csv')

In [6]:
df_dominant_topic

,Unnamed: 0,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0,7.0,0.1450,"industry, business, company, market, governmen...","['month', 'episode', 'fisherman', 'constituenc..."
1,1,1,3.0,0.1806,"area, friend, problem, constituency, part, dev...","['contradicting', 'competition', 'route', 'com..."
2,2,2,2.0,0.1434,"member, friend, debate, point, house, party, o...","['year', 'fishing', 'vote', 'citizen', 'colour..."
3,3,3,6.0,0.1261,"bill, power, amendment, clause, provision, leg...","['question', 'amendment']"
4,4,4,3.0,0.1261,"area, friend, problem, constituency, part, dev...","['herd', 'gilt']"
...,...,...,...,...,...,...
9995,9995,9995,3.0,0.1624,"area, friend, problem, constituency, part, dev...","['motion', 'question', 'standing_order', 'prov..."
9996,9996,9996,5.0,0.1874,"government, country, school, state, education,...","['discussion', 'industry', 'stakeholder', 'opp..."
9997,9997,9997,0.0,0.1316,"people, time, work, number, day, job, week, th...","['friend', 'question', 'support', 'pupil', 'lo..."
9998,9998,9998,3.0,0.2832,"area, friend, problem, constituency, part, dev...","['friend', 'member', 'peterborough', 'governme..."


In [5]:
embedding

,Unnamed: 0,x,y,hue
0,0,4.386418,-6.397941,7
1,1,0.827167,-33.258675,3
2,2,37.151672,28.609045,2
3,3,19.902620,-37.134434,6
4,4,34.370880,-16.822489,3
...,...,...,...,...
9995,9995,-52.321537,6.045866,3
9996,9996,1.627764,26.701094,5
9997,9997,-33.535885,-25.832993,0
9998,9998,0.697376,-46.144863,3


In [10]:
import pickle
objects = []
with (open("/content/drive/MyDrive/topic9/topic_distribution.pkl", "rb")) as openfile:
    while True:
        try:
            objects.append(pickle.load(openfile))
        except EOFError:
            break

In [ ]:
objects

In [12]:
df_topic_distribution=pd.DataFrame(objects[0], columns=['doc', 'topic1', 'topic2', 'topic3','topic4', 'topic5','topic6', 'topic7','topic8', 'topic9'])

In [13]:
df_topic_distribution['x'] = embedding['x']
df_topic_distribution['y'] = embedding['y']
df_topic_distribution['ref'] = df['id']
df_topic_distribution['speech'] = df['speech']
df_topic_distribution['speaker'] = df['display_as']
df_topic_distribution['location'] = df['constituency']
df_topic_distribution['party'] =df['party']
df_topic_distribution['date'] = df['date']
import datetime
def clean_date(date):
  datem = datetime.datetime.strptime(date, "%Y-%m-%d")
  return datem.year
df_topic_distribution['clean_date'] = df_topic_distribution['date'].apply(clean_date)
df_topic_distribution

,doc,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,x,y,ref,speech,speaker,location,party,date,clean_date
0,0,0.128060,0.128060,0.116761,0.120527,0.094162,0.112994,0.097928,0.096045,0.105461,4.386418,-6.397941,uk.org.publicwhip/debate/1979-11-27a.1096.7,Is the Minister aware that in the last two mon...,Robert Maclennan,Caithness and Sutherland,Labour,1979-11-27,1979
1,1,0.152778,0.086806,0.123264,0.098958,0.109375,0.137153,0.090278,0.098958,0.102431,0.827167,-33.258675,uk.org.publicwhip/debate/1979-11-27a.1136.2,Is not the right hon. Gentleman contradicting ...,Roger Moate,Faversham,Conservative,1979-11-27,1979
2,2,0.121864,0.107527,0.103943,0.132616,0.111111,0.105735,0.094982,0.096774,0.125448,37.151672,28.609045,uk.org.publicwhip/debate/1979-11-29a.1477.7,Does the right hon. Gentleman recall that he s...,James Lamond,Oldham East,Labour,1979-11-29,1979
3,3,0.106838,0.106838,0.106838,0.106838,0.106838,0.106838,0.106838,0.126068,0.126068,19.902620,-37.134434,uk.org.publicwhip/debate/1979-11-12a.1080.3,"Question put, That the amendment be made:",Unknown,NaN,NaN,1979-11-12,1979
4,4,0.106838,0.106838,0.108974,0.126068,0.106838,0.106838,0.121795,0.106838,0.108974,34.370880,-16.822489,uk.org.publicwhip/debate/1979-11-29a.1470.2,At a most recent census in August 1979 the Uni...,Alick Buchanan-Smith,North Angus and Mearns,Conservative,1979-11-29,1979
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.095726,0.085470,0.116239,0.104274,0.104274,0.085470,0.116239,0.194872,0.097436,-52.321537,6.045866,uk.org.publicwhip/debate/2021-02-24a.1024.2,"Motion made, and Question put forthwith (Stand...",Unknown,NaN,NaN,2021-02-24,2021
9996,9996,0.096186,0.106136,0.116086,0.142620,0.082919,0.092869,0.117745,0.082919,0.162521,1.627764,26.701094,uk.org.publicwhip/debate/2021-04-28c.361.4,I have regular discussions with ministerial co...,David Duguid,Banff and Buchan,Conservative,2021-04-28,2021
9997,9997,0.104274,0.167521,0.114530,0.105983,0.090598,0.094017,0.095726,0.085470,0.141880,-33.535885,-25.832993,uk.org.publicwhip/debate/2021-02-22b.655.7,I am really grateful to my hon. Friend for his...,Boris Johnson,Uxbridge and South Ruislip,Conservative,2021-02-22,2021
9998,9998,0.060215,0.098925,0.150538,0.234409,0.067384,0.091756,0.156989,0.050179,0.089606,0.697376,-46.144863,uk.org.publicwhip/debate/2021-02-04b.1186.0,I welcome this debate and thank my hon. Friend...,Robert Halfon,Harlow,Conservative,2021-02-04,2021


In [14]:
result= pd.concat([df_topic_distribution, df_dominant_topic], axis=1)
result.to_csv('/content/drive/MyDrive/topic9/version.csv')
result

,doc,topic1,topic2,topic3,topic4,topic5,topic6,topic7,topic8,topic9,...,location,party,date,clean_date,Unnamed: 0,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,0.128060,0.128060,0.116761,0.120527,0.094162,0.112994,0.097928,0.096045,0.105461,...,Caithness and Sutherland,Labour,1979-11-27,1979,0,0,7.0,0.1450,"industry, business, company, market, governmen...","['month', 'episode', 'fisherman', 'constituenc..."
1,1,0.152778,0.086806,0.123264,0.098958,0.109375,0.137153,0.090278,0.098958,0.102431,...,Faversham,Conservative,1979-11-27,1979,1,1,3.0,0.1806,"area, friend, problem, constituency, part, dev...","['contradicting', 'competition', 'route', 'com..."
2,2,0.121864,0.107527,0.103943,0.132616,0.111111,0.105735,0.094982,0.096774,0.125448,...,Oldham East,Labour,1979-11-29,1979,2,2,2.0,0.1434,"member, friend, debate, point, house, party, o...","['year', 'fishing', 'vote', 'citizen', 'colour..."
3,3,0.106838,0.106838,0.106838,0.106838,0.106838,0.106838,0.106838,0.126068,0.126068,...,NaN,NaN,1979-11-12,1979,3,3,6.0,0.1261,"bill, power, amendment, clause, provision, leg...","['question', 'amendment']"
4,4,0.106838,0.106838,0.108974,0.126068,0.106838,0.106838,0.121795,0.106838,0.108974,...,North Angus and Mearns,Conservative,1979-11-29,1979,4,4,3.0,0.1261,"area, friend, problem, constituency, part, dev...","['herd', 'gilt']"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,9995,0.095726,0.085470,0.116239,0.104274,0.104274,0.085470,0.116239,0.194872,0.097436,...,NaN,NaN,2021-02-24,2021,9995,9995,3.0,0.1624,"area, friend, problem, constituency, part, dev...","['motion', 'question', 'standing_order', 'prov..."
9996,9996,0.096186,0.106136,0.116086,0.142620,0.082919,0.092869,0.117745,0.082919,0.162521,...,Banff and Buchan,Conservative,2021-04-28,2021,9996,9996,5.0,0.1874,"government, country, school, state, education,...","['discussion', 'industry', 'stakeholder', 'opp..."
9997,9997,0.104274,0.167521,0.114530,0.105983,0.090598,0.094017,0.095726,0.085470,0.141880,...,Uxbridge and South Ruislip,Conservative,2021-02-22,2021,9997,9997,0.0,0.1316,"people, time, work, number, day, job, week, th...","['friend', 'question', 'support', 'pupil', 'lo..."
9998,9998,0.060215,0.098925,0.150538,0.234409,0.067384,0.091756,0.156989,0.050179,0.089606,...,Harlow,Conservative,2021-02-04,2021,9998,9998,3.0,0.2832,"area, friend, problem, constituency, part, dev...","['friend', 'member', 'peterborough', 'governme..."
